In [2]:
from Myo_Uplift_PDF import PDF
from Myo_Uplift_PDF import report_data

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pylab import rcParams
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
rcParams['figure.figsize'] = 12,4
rcParams['savefig.dpi'] = 300
from tqdm import tqdm
logo_path = 'logo/'

now = datetime.now().strftime('%Y%m%d')

day = 20240106

myo = sorted(glob(f'Data/{day}/myomotion/*csv'))
mname = [f"{os.path.basename(i).split('_')[0]}_{os.path.basename(i).split('_')[1]}" for i in myo]

upli = sorted(glob(f'Data/{day}/uplift/*csv'))
uname = [os.path.basename(i).replace('.csv','') for i in upli]

comment_path = f'Data/{day}/comment/'
mname, uname, comment_path

/Users/yongseok/opt/anaconda3/envs/ML/lib/python3.11/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


(['Jeonseongwoo_006', 'Kimjieon_008', 'Kwonhyukchan_009', 'Leejinyong_011'],
 ['jeonseongwoo_006', 'kimjieon_008', 'kwonhyukchan_009', 'leejinyong_011'],
 'Data/20240106/comment/')

In [3]:
myomotion = {}
for file, name in zip(myo,mname):
    myomotion[name] = file

uplift = {}
for file, name in zip(upli,mname):
    uplift[name] = file

In [4]:
uplift

{'Jeonseongwoo_006': 'Data/20240106/uplift/jeonseongwoo_006.csv',
 'Kimjieon_008': 'Data/20240106/uplift/kimjieon_008.csv',
 'Kwonhyukchan_009': 'Data/20240106/uplift/kwonhyukchan_009.csv',
 'Leejinyong_011': 'Data/20240106/uplift/leejinyong_011.csv'}

In [5]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        self.cell(0, -10, 'Kookmin Univ. Hitting Report', ln = True)

    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'{logo_path}/logo.png',
                   160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(0, 128, 0)
        self.set_text_color(255, 255, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
    'pelvis_rotational_velocity_with_respect_to_ground'  : ['Pelvic', 'r'],
    'trunk_rotational_velocity_with_respect_to_ground'   : ['Torso', 'g'],
    'lead_arm_rotational_velocity_with_respect_to_ground': ['Upper Arm','b'],
}

one_cols = {
    # 'trunk_twist_clockwise'                  : 'X FACTOR',
    'trunk_lateral_flexion'                  : 'TRUNK LATERAL TILT',
    'rear_shoulder_adduction'                : 'REAR SHOULDER AD-ABDUCTION',
    'lead_knee_extension'                    : 'LEAD KNEE FLEXION-EXTENSION',
    'lead_knee_extension_velocity'           : 'LEAD KNEE EXTENSION ANGULAR VELOCITY',
    'lead_hip_flexion_with_respect_to_trunk' : 'LEAD HIP FLEXION-EXTENSION',
    'shank_angle'                            : 'SHANK ANGLE',
    'hip_elbow_loss_space'                   : 'HIP-ELBOW DISTANCE',
    'shoulder_hand_loss_space'               : 'SHOULDER-HAND DISTANCE',
    'hand_shoulder_distance'                 : 'SHOULDER-HAND DISTANCE (AP)',
    'lead_elbow_flexion'                     : 'ELBOW FLEXION',
    'rear_shoulder_adduction_velocity'       : 'REAR SHOULDER AB-ADDUCTION VELOCITY',
    'lead_elbow_flexion_velocity'            : 'LEAD ELBOW EXTENSION VELOCITY',
    'pelvis_rotational_velocity_with_respect_to_ground' : 'PELVIS ROTATION VELOCITY'
}

lead_cols= {
    'lead_shoulder_external_rotation' : ['SHOULDER', 'darkblue'],
    'lead_elbow_flexion'              : ['ELBOW'   , 'firebrick'],
}

elbow_cols = {
    'rear_elbow_jc_3d_y' : ['REAR ELBOW', 'darkblue'],
    'lead_elbow_jc_3d_y' : ['LEAD ELBOW', 'firebrick'],
}

for_one_cols = {
    'lead_force_z' : 'LEAD LEG GROUND REACTION FORCE (Vertical)',
    'rear_force_z' : 'REAR LEG GROUND REACTION FORCE (Vertical)',
    'rear_force_x' : 'REAR LEG AP GRF',
    'lead_moment_z': 'LEAD LEG TORQUE',
    'rear_moment_z': 'REAR LEG TORQUE'
}

grf_y_cols = {
    'rear_force_x' : ['Rear Leg', 'b'],
    'lead_force_y' : ['Lead Leg', 'r']
}

for file in tqdm(myomotion):
    uplift_df = pd.read_csv(uplift[file])
    myo_df = pd.read_csv(myomotion[file])
    
    uplift_to_time = int(uplift_df['to_frame'].values[0])
    uplift_kh_time = int(uplift_df['kh_frame'].values[0] - uplift_to_time)
    uplift_fc_time = int(uplift_df['fc_frame'].values[0] - uplift_to_time)
    uplift_bc_time = int(uplift_df['bc_frame'].values[0])
    uplift_pel_initial_time = int(uplift_df['pelvis_rotation_initial'][0] - uplift_to_time)
    uplift_df = uplift_df.iloc[uplift_to_time:uplift_bc_time+60].reset_index(drop=True)
    
    uplift_bc_time = int(uplift_bc_time - uplift_to_time)
    uplift_to_time = 0
    uplift_time = uplift_df['time']
    uplift_total_time = uplift_bc_time - uplift_fc_time
    
    myo_time = myo_df['time']
    myo_to_time = myo_df['to_time'][0]
    myo_fc_time = myo_df['fc_10_time'][0]
    max_grf_time = myo_df['lead_max_grf_time'][0]
    myo_bc_time = myo_df['bc_time'][0]
    peak_hand_time = myo_df['peak_hand_time'][0]
    myo_total_time = myo_bc_time - myo_fc_time 
    
    store = f'Data/{day}/report'
    player = file.split('_')[0]
    weight = uplift_df['weight'][0]
    height = uplift_df['height'][0]
    side = uplift_df['handedness'][0]
    stride = round(uplift_df['stride_length'][0], 2)
    
    with open(f"{comment_path}/{file.split('_')[0]}_comment.txt", "rb") as fh:
        comment_txt = fh.read().decode('utf-8')
        
    position_kinematic = comment_txt.find('- KINEMATIC SEQUENCE')
    position_loading = comment_txt.find('- LOADING PHASE')
    position_stride = comment_txt.find('- STRIDE PHASE')
    position_daed_hands = comment_txt.find('- DEAD HANDS [LEAD SHOULDER - HAND DISTANCE]')
    position_casting_hand = comment_txt.find('- CASTING THE HANDS')
    position_loss_space_1 = comment_txt.find('- LOSS OF SPACE 1')
    position_loss_space_2 = comment_txt.find('- LOSS OF SPACE 2')
    position_loss_space_3 = comment_txt.find('- LOSS OF SPACE 3')
    position_trail_grf = comment_txt.find('- Trail Leg GRF')
    position_drive_grf = comment_txt.find('- Drive Leg GRF')
    position_grf_y = comment_txt.find('- REAR & LEAD LEG GROUND REACTION FORCE')
    position_rear_arm = comment_txt.find('- REAR SHOULDER AB/ADDUCTION ANGULAR VELOCITY')
    position_lead_knee_ext_vel = comment_txt.find('- LEAD LEG KNEE EXTENSION ANGULAR VELOCITY')
    position_lead_elb_ext_vel = comment_txt.find('- LEAD ELBOW EXTENSION ANGULAR VELOCITY')
    position_trunk = comment_txt.find('- TRUNK LATERAL TILT')
    
    comment_kinematic = comment_txt[position_kinematic:position_loading]
    comment_loading = comment_txt[position_loading:position_stride]
    comment_stride = comment_txt[position_stride:position_daed_hands]
    comment_dead_hands = comment_txt[position_daed_hands:position_casting_hand]
    comment_casting_hand = comment_txt[position_casting_hand:position_loss_space_1]
    comment_loss_space_1 = comment_txt[position_loss_space_1:position_loss_space_2]
    comment_loss_space_2 = comment_txt[position_loss_space_2:position_loss_space_3]
    comment_loss_space_3 = comment_txt[position_loss_space_3:position_trail_grf]
    comment_trail_grf = comment_txt[position_trail_grf:position_drive_grf]
    comment_drive_grf = comment_txt[position_drive_grf:position_grf_y]
    comment_grf_y = comment_txt[position_grf_y:position_rear_arm]
    comment_rear_arm = comment_txt[position_rear_arm:position_lead_knee_ext_vel]
    comment_lead_knee_ext_vel = comment_txt[position_lead_knee_ext_vel:position_lead_elb_ext_vel]
    comment_lead_elb_ext_vel = comment_txt[position_lead_elb_ext_vel:position_trunk]
    comment_trunk = comment_txt[position_trunk:]

    
    ks = report_data.kinematic_sequence(uplift_df,ks_cols, uplift_time, uplift_to_time, uplift_kh_time, uplift_fc_time, uplift_bc_time)
    ang = report_data.one_angle(uplift_df,one_cols, uplift_time, uplift_to_time, uplift_kh_time, uplift_fc_time, uplift_bc_time,uplift_pel_initial_time)
    arm = report_data.lead_arm(uplift_df,lead_cols, uplift_time, uplift_to_time, uplift_kh_time, uplift_fc_time, uplift_bc_time)
    elb = report_data.elbow_height(uplift_df,elbow_cols, uplift_time, uplift_to_time, uplift_kh_time, uplift_fc_time, uplift_bc_time)
    y = report_data.grf_y(myo_df, grf_y_cols, myo_time, myo_to_time, myo_fc_time, max_grf_time, myo_bc_time)
    rear, lead_tq, lead_z,rear_grf_y = report_data.one_force(myo_df, for_one_cols, myo_time, myo_to_time, myo_fc_time, max_grf_time, myo_bc_time)
    xf = report_data.xfactor(myo_df, 'x_factor', myo_time, myo_to_time, myo_fc_time, max_grf_time, myo_bc_time)
    
    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)
    # ===========================================================================================================
    # main page
    pdf.add_page()
    # pdf.set_fill_color(r=150,g=0,b=0)
    # pdf.rect(w=210, h=297, x=0, y=0, style="DF")
    pdf.set_font('helvetica', 'BU', 20)
    title = 'Kookmin Univ. Hitting Report'
    pdf.ln(10)
    title_w = pdf.get_string_width(title)
    doc_w = pdf.w
    pdf.set_x(title_w/2)
    pdf.cell(135, 20, title, align = 'C', ln = True)

    # ===========================================================================================================
    # chapter
    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()
    ch9_link = pdf.add_link()
    ch10_link = pdf.add_link()
    ch11_link = pdf.add_link()
    ch12_link = pdf.add_link()
    ch13_link = pdf.add_link()
    ch14_link = pdf.add_link()
    ch15_link = pdf.add_link()
    ch16_link = pdf.add_link()
    ch17_link = pdf.add_link()
    ch18_link = pdf.add_link()
    
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0, 2,"",ln=1,align = 'C')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}   Hitter Type : {side}   Weight : {weight}    Height : {height}", align = 'C', ln = True)
    pdf.cell(0, 5,"",ln=1,align = '')
    pdf.image(f'{logo_path}/analysis.png',x= 25,w=160,h=65)
    
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.set_fill_color(r=150,g=0,b=0)
    pdf.set_text_color(255,255,255)
    pdf.cell(0,6, 'SWING EFFICIENCY', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.set_font('helvetica', 'B', 11)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(100,5, '1. KINEMATIC SEQUENCE', ln = 1, align = '', link = ch1_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '2. PELVIC, TRUNK, ARM ANGULAR VELOCITY', ln = True, align = '', link = ch4_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '3. PEAK ANGULAR VELOCITY TIMING', ln = True, align = '', link = ch4_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '4. SPEED GAIN', ln = True, align = '', link = ch4_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '5. STRIDE LENGTH', ln = True, align = '', link = ch4_link)
    pdf.cell(0,3,"",ln=1,align = '')
    
    pdf.set_font('helvetica', 'B', 13)
    pdf.set_text_color(255,255,255)
    pdf.cell(90,6, 'LOADING PHASE', ln = False, align = 'C', fill=True,link = ch2_link)
    pdf.set_x(110)
    pdf.cell(90,6, 'STRIDE PHASE', ln = True, align = 'C', fill=True,link = ch4_link)
    pdf.set_font('helvetica', 'B', 11)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(100,5, '1. SWAY', ln = False, align = '', link = ch2_link)
    pdf.set_x(110)
    pdf.cell(0,5, '1. LOSS OF POSTURE', ln = True, align = '', link = ch4_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(100,5, '2. REAR LEG TORQUE', ln = False, align = '', link = ch3_link)
    pdf.set_x(110)
    pdf.cell(100,5, '2. REAR LEG AP GRF', ln = True, align = '', link = ch5_link)
    pdf.cell(0,3,"",ln=1,align = '')

    pdf.set_font('helvetica', 'B', 13)
    pdf.set_text_color(255,255,255)
    pdf.cell(0,6, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch6_link)
    pdf.set_font('helvetica', 'B', 11)
    pdf.cell(0,3,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(0,5, '1. DEAD HANDS', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '2. CASTING THE HANDS', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '3. LOSS OF SPACE 1 [PELVIS OPEN TIMING]', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '4. LOSS OF SPACE 2 [LEAD SHOULDER-HAND DISTANCE]', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '5. LOSS OF SPACE 3 [ELBOW HEIGHT]', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '6. REAR LEG VERTICAL GRF', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '7. REAR LEG TORQUE', ln = 1, align = '', link = ch11_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '8. LEAD LEG VERTICAL GRF', ln = 1, align = '', link = ch12_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '9. LEAD LEG TORQUE', ln = 1, align = '', link = ch13_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '10. REAR & LEAD LEG AP GRF [X-Axis Timing]', ln = 1, align = '', link = ch14_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '11. REAR SHOULDER AB/ADDUCTION VELOCITY', ln = 1, align = '', link = ch15_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '12. LEAD LEG KNEE EXTENSION VELOCITY', ln = 1, align = '', link = ch16_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '13. LEAD ARM PEAK ELBOW EXTENSION VELOCITY', ln = 1, align = '', link = ch17_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0,5, '14. TRUNK TILT', ln = 1, align = '', link = ch18_link)
    
    # pdf.set_font('helvetica', 'B', 15)
    # pdf.cell(0,15,"",ln=1,align = '')
    # pdf.set_text_color(255,255,255)
    # pdf.cell(0,7, 'FEEDBACK', ln = 1, align = 'C',fill=True, link = ch11_link)
    # pdf.set_text_color(0,0,0)

    # ===============================================================================================
    peak_pel = ks['max']['pelvis_rotational_velocity_with_respect_to_ground']; time_pel = ks['time']['pelvis_rotational_velocity_with_respect_to_ground']
    peak_tor = ks['max']['trunk_rotational_velocity_with_respect_to_ground'];time_tor = ks['time']['trunk_rotational_velocity_with_respect_to_ground']
    peak_upper = ks['max']['lead_arm_rotational_velocity_with_respect_to_ground'];time_upper = ks['time']['lead_arm_rotational_velocity_with_respect_to_ground']
    # peak_fore = ks['max']['lead_shoulder_external_rotation_velocity'];time_fore = ks['time']['lead_shoulder_external_rotation_velocity']

    pel_time = round(100 * (time_pel - uplift_fc_time) / uplift_total_time, 2)
    tor_time = round(100 * (time_tor - uplift_fc_time) / uplift_total_time, 2)
    upper_time = round(100 * (time_upper - uplift_fc_time) / uplift_total_time, 2)
    # fore_time = round(100 * (time_fore - fc_time) / total_time, 2)
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_upper / peak_tor,2)
    # fore_gain = round(peak_fore / peak_upper,2)

    pdf.add_page()
    pdf.set_link(ch1_link, page = 2)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING EFFICIENCY', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'KINEMATIC SEQUENCE', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The order and velocity of the pelvis, trunk and arm rotation')
    pdf.image(f'figure/KINEMATIC SEQUENCE.png',x=75,y=28,w=133,h=50)
    pdf.image(f'{logo_path}/kinematic.png',x=17,y=28,w=50,h=55)
    pdf.set_font('helvetica', '', 9)
    pdf.set_x(20)
    pdf.set_y(83)
    pdf.set_text_color(150,0,0)
    pdf.cell(0,5, f"Stride Length : {stride} [% Height]", ln = 1, align = '')
    pdf.set_text_color(0,0,0)
    data_as_dict = {"Segment"                 : ["Pelvic"       , "Torso"        ,"Arm"            ],
                    "Pro Range"               : ["300 - 600"    , "600 - 800"    ,"850 - 2000"    ],
                    "Peak Velocity"           : [peak_pel       , peak_tor       ,peak_upper       ],
                    "Timing [BC - FC (100%)]" : [f"{pel_time} %", f"{tor_time} %",f"{upper_time} %"],
                    "Speed Gain"              : [''             , tor_gain       ,upper_gain       ]}
    
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Kinematic Sequence',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [30,40,35,55,40],
                    x_start='C')
    
    pdf.set_font('helvetica', '', 9)
    pdf.set_y(85)
    pdf.set_text_color(150,0,0)
    pdf.cell(0,7,"",ln=1,align = '')
    pdf.cell(0,5, f"Pelvis Initial Rotation Time [BC - FC (100%)] : {round(100 * (uplift_pel_initial_time - uplift_fc_time) / uplift_total_time,2)} %", ln = 1, align = 'R')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(150)
    pdf.set_font('kor', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, comment_kinematic, ln = True, border='True')

    # ===========================================================================================================
    # ===========================================================================================================
    # Shank Angle
    pdf.add_page()
    pdf.set_link(ch2_link, page = 3)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'LOADING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SWAY [SHANK ANGLE]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The angle of the longitudinal axis of the shank to the vertical axis in the sagittal plane')
    pdf.image('figure/SHANK ANGLE.png',x=75,y=25,w=133,h=50)
    pdf.image(f'{logo_path}/shank_angle.png',x=15,y=25,w=50,h=50)

    data_as_dict = {"":["Pro","Me"],
                    "at Lead Leg Toe Off" : ['7.53 ± 5.70',ang['to_time']['shank_angle']],
                    "at Knee High"        : ['-',ang['kh_time']['shank_angle']],
                    "at Fooc Contact"     : ['25.06 ± 7.72',ang['fc_time']['shank_angle']],
                    "at Ball Contact"     : ['69.96 ± 6.25',ang['bc_time']['shank_angle']]
                    }

    pdf.set_y(76)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro","Me"],
                    "Min"        : ['4.10 ± 4.85',ang['min']['shank_angle']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['shank_angle'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(99)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    # pdf.set_y(158)
    # pdf.set_font('kor', 'B', 12)
    # pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    # pdf.set_font('kor', 'B', 9)
    # pdf.set_y(163)
    # pdf.multi_cell(0, 8, shank_ang_comment, ln = True, border='True')
    # ===========================================================================================================
    # Rear Leg Torque
    # pdf.add_page()
    pdf.set_link(ch2_link, page = 3)
    # pdf.set_y(15)
    pdf.set_y(125)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR LEG TORQUE', ln = True)
    # pdf.set_y(21)
    pdf.set_y(130)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The rear leg torque relative to the ground')
    pdf.image(f'figure/rear_moment_z.png',x=75,y=132,w=133,h=50)
    pdf.image(f'{logo_path}/rear_leg_torque.png',x=17,y=132,w=50,h=50)

    data_as_dict = {
                    "Max at Loading Phase" : [rear['max_at_loading']['rear_moment_z']],
                    "at Lead Leg Toe Off"  : [rear['to_time']['rear_moment_z']],
                    "at Foot Contact"     : [rear['fc_time']['rear_moment_z']],
                    "at Max GRF"          : [rear['max_grf_time']['rear_moment_z']],
                    "at Ball Contact"     : [rear['bc_time']['rear_moment_z']]
                    }
    pdf.set_y(180)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Rear Leg Torque [Proficient = 45 Nm]',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')
    
    pdf.set_y(207)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0, 0, 'Loding Phase Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 8)
    pdf.set_y(210)
    pdf.multi_cell(0, 8, comment_loading, ln = True, border='True')
    # =============================================================================
    # Trunk Lataral Tilt [Loss of Posture]
    pdf.add_page()
    pdf.set_link(ch3_link, page = 4)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'STRIDE PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF POSTURE [TRUNK LATERAL TILT]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Frontal plane rotation of the trunk relative to the pelvis')
    pdf.image('figure/TRUNK LATERAL TILT.png',x=75,y=25,w=133,h=50)
    pdf.image(f'{logo_path}/lateral tilt.png',x=8,y=25,w=65,h=50)

    data_as_dict = {""                    : ["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-4.77 ± 5.49', ang['to_time']['trunk_lateral_flexion']],
                    "at Knee High"        : ['-',ang['kh_time']['trunk_lateral_flexion']],
                    "at Fooc Contact"     : ['-8.17 ± 4.64',ang['fc_time']['trunk_lateral_flexion']],
                    "at Ball Contact"     : ['32.23 ± 6.40',ang['bc_time']['trunk_lateral_flexion']]
                    }

    pdf.set_y(70)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lateral Trunk Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"" :["Pro", "Me"],
                    "Max"        : ['34.77 ± 6.29',ang['max']['trunk_lateral_flexion']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['trunk_lateral_flexion'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(99)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    # pdf.set_y(158)
    # pdf.set_font('kor', 'B', 12)
    # pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    # pdf.set_font('kor', 'B', 9)
    # pdf.set_y(163)
    # pdf.multi_cell(0, 8, trunk_lat_tilt_comment, ln = True, border='True')
    # ========================================================================================
    # REAR LEG AP GRF
    # pdf.add_page()
    pdf.set_link(ch3_link, page = 4)
    # pdf.set_y(15)
    pdf.set_y(125)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR LEG AP GRF', ln = True)
    # pdf.set_y(21)
    pdf.set_y(130)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The ground reaction force of rear leg toward to pitchers')
    pdf.image(f'figure/rear_force_x.png',x=75,y=132,w=133,h=50)
    pdf.image(f'{logo_path}/rear_lead_y.png',x=17,y=132,w=50,h=50)

    data_as_dict = {

                    "Max at Loading Phase" : [
                                            # '41.37 ± 9.65',
                                             rear_grf_y['max']],
                    "Max Timing"       : [
                                            # '-'           ,
                                            f"{round(100 * (rear_grf_y['max_timing']-myo_fc_time)/ myo_total_time , 2)} %"],
                    }
    # pdf.set_y(90)
    pdf.set_y(180)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Rear Leg AP Ground Reaction Force',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')

    # pdf.set_y(150)
    pdf.set_y(207)
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0, 0, 'Stride Phase Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 8)
    pdf.set_y(210)
    # pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_stride, ln = True, border='True')

    # ===================================================================================
    # LEAD SHOULDER - LEAD ELBOW DISTANCE 앞뒤 방향만 고려 [DEAD HANDS]
    pdf.add_page()
    pdf.set_link(ch4_link, page = 5)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'DEAD HANDS [LEAD SHOULDER - HAND DISTANCE]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The distance between the lead shoulder and lead hand')
    pdf.image(f'figure/SHOULDER-HAND DISTANCE (AP).png',x=75,y=26,w=133,h=50)
    pdf.image(f'{logo_path}/shoulder_hand_ap.png',x=17,y=26,w=50,h=50)

    data_as_dict = {"" : ["Pro", "Me"],
                    "at Lead Leg Toe Off" : ['-0.32 ± 0.04',ang['to_time']['hand_shoulder_distance']],
                    "at Knee High"        : ['-',ang['kh_time']['hand_shoulder_distance']],
                    "at Fooc Contact"     : ['-0.47 ± 0.03',ang['fc_time']['hand_shoulder_distance']],
                    "at Ball Contact"     : ['0.43 ± 0.03',ang['bc_time']['hand_shoulder_distance']]
                    }
    pdf.set_y(76)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip - Elbow Loss of Space',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro", "Me"],
                    "Min"        : ['-0.37 ± 0.04',ang['min']['hand_shoulder_distance']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['hand_shoulder_distance'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(105)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')  
    
    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_dead_hands, ln = True, border='True')   
    # ===========================================================================================================
    # Casting the Hand [Elbow Flexion]
    pdf.add_page()
    pdf.set_link(ch5_link, page = 6)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
        
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'CASTING THE HANDS [Elbow Flexion]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The distance between lead shoulder and lead hand')
    pdf.image(f'figure/ELBOW FLEXION.png',x=75,y=32,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_flexion.png',x=17,y=32,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['lead_elbow_flexion']],
                    "at Knee High"        : [ang['kh_time']['lead_elbow_flexion']],
                    "at Fooc Contact"     : [ang['fc_time']['lead_elbow_flexion']],
                    "at Ball Contact"     : [ang['bc_time']['lead_elbow_flexion']],
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Check the Dead Hands',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')
    
    data_as_dict = {
                    "at Max"            : [ang['max']['lead_elbow_flexion']],
                    "Max Timing"        : [f"{round(100 * (ang['max_time']['lead_elbow_flexion'] - uplift_fc_time) / uplift_total_time)} %"]
                    }
    pdf.set_y(112)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')    
    

    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_casting_hand, ln = True, border='True')
    
      # ===================================================================================
    # PELVIS OPEN TIMING [Loss of Space 1]
    pdf.add_page()
    pdf.set_link(ch6_link, page = 7)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF SPACE 1 [PELVIS OPEN TIMING]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The distance between the lead shoulder and lead hand')
    pdf.image(f'figure/PELVIS ROTATION VELOCITY.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/pel_rot_vel.png',x=17,y=30,w=50,h=50)


    data_as_dict = {"":["Pro", "Me"],
                    "Pelvic Open Timing"        : ["-22.29 ± 18.03 %",f"{round(100 * (uplift_pel_initial_time - uplift_fc_time) / uplift_total_time,2)} %"],
                    "MAX"                       : ["300 - 600"       ,ang['max']['pelvis_rotational_velocity_with_respect_to_ground']],
                    "Max Timing"                : ["-"               ,f"{pel_time} %"],
                    }
    
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip - Elbow Loss of Space',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50,],
                    x_start='C')

    
    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_loss_space_1, ln = True, border='True')  
     # ===================================================================================
    # LEAD SHOULDER - LEAD ELBOW 절대거리 [Loss of Space]
    pdf.add_page()
    pdf.set_link(ch7_link, page = 8)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF SPACE 2 [LEAD SHOULDER - HAND DISTANCE]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The distance between the lead shoulder and lead hand')
    pdf.image(f'figure/SHOULDER-HAND DISTANCE.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/shoulder_hand_loss.png',x=17,y=30,w=50,h=50)

    data_as_dict = {"" : ["Pro", "Me"],
                    "at Lead Leg Toe Off" : ['-0.32 ± 0.04',ang['to_time']['shoulder_hand_loss_space']],
                    "at Knee High"        : ['-',ang['kh_time']['shoulder_hand_loss_space']],
                    "at Fooc Contact"     : ['-0.47 ± 0.03',ang['fc_time']['shoulder_hand_loss_space']],
                    "at Ball Contact"     : ['0.43 ± 0.03',ang['bc_time']['shoulder_hand_loss_space']]
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip - Elbow Loss of Space',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro", "Me"],
                    "Min"        : ['-0.37 ± 0.04',ang['min']['shoulder_hand_loss_space']],
                    "Min Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['min_time']['shoulder_hand_loss_space'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')  
    
    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_loss_space_2, ln = True, border='True')  
    # ===========================================================================================================
    # ELBOW HEIGHT
    pdf.add_page()
    pdf.set_link(ch8_link, page = 9)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LOSS OF SPACE 2 [ELBOW HEIGHT]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The height of the lead and rear elbow')
    pdf.image('figure/elbow_height.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_height.png',x=8,y=25,w=65,h=60)

    data_as_dict = {
                    "Segment"             : ["Rear Elbow"                        , "Lead Elbow"                       ,"My Displacement"    ,"Pro Displacement"                                             ],
                    "at Lead Leg Toe Off" : [elb['to_time']['rear_elbow_jc_3d_y'],elb['to_time']['lead_elbow_jc_3d_y'],round(elb['to_time']['rear_elbow_jc_3d_y']-elb['to_time']['lead_elbow_jc_3d_y'],2), '0.13 ± 0.06'],
                    "at Knee High"        : [elb['kh_time']['rear_elbow_jc_3d_y'],elb['kh_time']['lead_elbow_jc_3d_y'],round(elb['kh_time']['rear_elbow_jc_3d_y']-elb['kh_time']['lead_elbow_jc_3d_y'],2), '-'],
                    "at Fooc Contact"     : [elb['fc_time']['rear_elbow_jc_3d_y'],elb['fc_time']['lead_elbow_jc_3d_y'],round(elb['fc_time']['rear_elbow_jc_3d_y']-elb['fc_time']['lead_elbow_jc_3d_y'],2), '0.12 ± 0.08'],
                    "at Ball Contact"     : [elb['bc_time']['rear_elbow_jc_3d_y'],elb['bc_time']['lead_elbow_jc_3d_y'],round(elb['bc_time']['rear_elbow_jc_3d_y']-elb['bc_time']['lead_elbow_jc_3d_y'],2), '0.21 ± 0.10'],
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Elbow Height (Loss of Space)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    pdf.set_y(155)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(160)
    pdf.multi_cell(0, 8, comment_loss_space_3, ln = True, border='True')
    # data_as_dict = {
    #                 "Segment"                     : ["Rear Elbow"                    , "Lead Elbow"],
    #                 "Max"                         : [elb['max']['rear_elbow_jc_3d_y'],elb['max']['lead_elbow_jc_3d_y'] ],
    #                 "Max Timing [BC - FC (100%)]" : [f"{round(100 * (elb['max_time']['rear_elbow_jc_3d_y'] - fc_time) / total_time,2)} %",f"{round(100 * (elb['max_time']['lead_elbow_jc_3d_y'] - fc_time) / total_time,2)} %" ],
    #                 }
    # pdf.set_y(118)
    # title_size = 8
    # pdf.set_font(family='Arial', style='B', size=title_size)
    # pdf.create_table(table_data = data_as_dict,
    #                 title = '',
    #                 title_size=10,
    #                 align_header = 'C',
    #                 align_data = 'C',
    #                 cell_width = [64,64,64],
    #                 x_start='C')
    
    # ===========================================================================================================
    # Rear Leg GRF Z
    pdf.add_page()
    pdf.set_link(ch9_link, page = 10)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR LEG GROUND REACTION FORCE (Vertical)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The ground reaction force vertical to the ground')
    pdf.image(f'figure/rear_force_z.png',x=75,y=25,w=133,h=50)
    pdf.image(f'{logo_path}/rear_leg_z.png',x=17,y=25,w=50,h=50)

    data_as_dict = {
                    "Max at Loading Phase" : [rear['max_at_loading']['rear_force_z']],
                    "at Lead Leg Toe Off"  : [rear['to_time']['rear_force_z']],
                    "at Foot Contact"     : [rear['fc_time']['rear_force_z']],
                    "at Max GRF"          : [rear['max_grf_time']['rear_force_z']],
                    "at Ball Contact"     : [rear['bc_time']['rear_force_z']]
                    }
    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (Vertical) [Proficient = above 98 %BW ]',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C') 

    # pdf.set_y(150)
    # pdf.set_font('helvetica', 'B', 12)
    # pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    # pdf.set_font('kor', 'B', 9)
    # pdf.set_y(155)
    # pdf.multi_cell(0, 8, rear_grf_comment, ln = True, border='True')
    
    # ===========================================================================================================
    # Rear Leg Torque
    # pdf.add_page()
    pdf.set_link(ch9_link, page = 10)
    pdf.set_y(100)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR LEG TORQUE', ln = True)
    pdf.set_y(105)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The rear leg torque relative to the ground')
    pdf.image(f'figure/rear_moment_z.png',x=75,y=110,w=133,h=50)
    pdf.image(f'{logo_path}/rear_leg_torque.png',x=17,y=110,w=50,h=50)

    data_as_dict = {
                    "Max at Loading Phase" : [rear['max_at_loading']['rear_moment_z']],
                    "at Lead Leg Toe Off"  : [rear['to_time']['rear_moment_z']],
                    "at Foot Contact"     : [rear['fc_time']['rear_moment_z']],
                    "at Max GRF"          : [rear['max_grf_time']['rear_moment_z']],
                    "at Ball Contact"     : [rear['bc_time']['rear_moment_z']]
                    }
    pdf.set_y(160)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Rear Leg Torque [Proficient = 45 Nm]',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')
    
    pdf.set_y(190)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Trail Leg GRF Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(195)
    pdf.multi_cell(0, 8, comment_trail_grf, ln = True, border='True')
    
    # ===========================================================================================================
    # Lead Leg GRF Z
    pdf.add_page()
    pdf.set_link(ch10_link, page = 11)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG GROUND REACTION FORCE (Vertical)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The ground reaction force vertical to ground')
    pdf.image(f'figure/lead_force_z.png',x=75,y=25,w=133,h=50)
    pdf.image(f'{logo_path}/lead_leg_z.png',x=17,y=25,w=50,h=50)

    data_as_dict = {
                    "at Foot Contact"     : [lead_z['fc_time']],
                    "Max"                 : [lead_z['max']],
                    "FC - MAX Time"       : [f"{lead_z['fc-max_time']} s"],
                    "Loading rate"        : [lead_z['loading_rate']]
                    }
    pdf.set_y(73)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Ground Reaction Force (Vertical)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')
    
    # pdf.set_y(150)
    # pdf.set_font('helvetica', 'B', 12)
    # pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    # pdf.set_font('kor', 'B', 9)
    # pdf.set_y(155)
    # pdf.multi_cell(0, 8, lead_grf_comment, ln = True, border='True')    
    # ===========================================================================================================
    # Lead Leg Torque
    # pdf.add_page()
    pdf.set_link(ch10_link, page = 11)
    pdf.set_y(100)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG TORQUE', ln = True)
    pdf.set_y(105)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The lead leg torque relative to the ground')
    pdf.image(f'figure/lead_moment_z.png',x=75,y=110,w=133,h=50)
    pdf.image(f'{logo_path}/lead_leg_torque.png',x=17,y=110,w=50,h=50)

    data_as_dict = {
                    "Max"             :  [lead_tq['max']],
                    "Max Timing [BC - FC (100%)]"      :  [f"{round(100 * (lead_tq['time'] - myo_fc_time) / myo_total_time,2)} %"],
                    "Good"            :  ['above 100 Nm']
                    }
    pdf.set_y(160)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Torque',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')

    pdf.set_y(190)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Drive Leg GRF Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(195)
    pdf.multi_cell(0, 8, comment_drive_grf, ln = True, border='True')
    
    # ===========================================================================================================
    # AP axis GRF Y
    pdf.add_page()
    pdf.set_link(ch11_link, page = 12)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR & LEAD LEG GROUND REACTION FORCE (AP axis)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The ground reaction force toward to pitcher')
    pdf.image(f'figure/grf_ap.png',x=75,y=28,w=133,h=50)
    pdf.image(f'{logo_path}/rear_lead_y.png',x=17,y=28,w=50,h=50)

    data_as_dict = {
                "Segment"             : ['Rear Leg','Lead Leg'],
                "Max"                 : [y['max']['rear_force_x'], y['max']['lead_force_y']],
                "at Lead Leg Max"     : [y['lead_max']['rear_force_x'], y['lead_max']['lead_force_y']],
                "Good"                : ['', 'above 70%BW']
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Read & Lead Leg AP aixs Ground Reaction Force',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                "X-Axis Timing"       : [f"{y['x_axis_time']} %"],
                "Proficient"          : ['above 80%BW']
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(155)
    pdf.multi_cell(0, 8, comment_grf_y, ln = True, border='True')
    # ===========================================================================================================
    # REAR SHOHULDER AD/ABDUCTION VELOCITY
    pdf.add_page()
    pdf.set_link(ch12_link, page = 13)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR SHOULDER AD/ABDUCTION ANGULAR VELOCITY', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Frontal plane rotation of the arm relative to the trunk')
    pdf.image('figure/REAR SHOULDER AB-ADDUCTION VELOCITY.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/add_vel.png',x=8,y=30,w=65,h=50)

    data_as_dict = {"" : ["Pro", "Me"],
                    "at Lead Leg Toe Off" : ['-30.53 ± 111.14',ang['to_time']['rear_shoulder_adduction_velocity']],
                    "at Knee High"        : ['-',ang['kh_time']['rear_shoulder_adduction_velocity']],
                    "at Fooc Contact"     : ['-27.20 ± 185.30',ang['fc_time']['rear_shoulder_adduction_velocity']],
                    "at Ball Contact"     : ['2.23 ± 145.85',ang['bc_time']['rear_shoulder_adduction_velocity']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Rear Shoulder Abduction',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"" : ["Pro", "Me"] ,
                    "Max"        : ['342.37 ± 185.09',ang['max']['rear_shoulder_adduction_velocity']],
                    "Max Timing [BC - FC (100%)]" : ["-", f"{round(100 * (ang['max_time']['rear_shoulder_adduction_velocity'] - uplift_fc_time) / uplift_total_time)} %"],
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_rear_arm, ln = True, border='True')

    # ===========================================================================================================
    # Lead Knee Flexion Velcoity
    pdf.add_page()
    pdf.set_link(ch13_link, page = 14)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Sagittal plane angular velocity of the shank relative to the thigh')
    pdf.image('figure/LEAD KNEE EXTENSION ANGULAR VELOCITY.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/knee vel.png',x=8,y=25,w=65,h=60)

    data_as_dict = {"":["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-70.52 ± 100.89',ang['to_time']['lead_knee_extension_velocity']],
                    "at Knee High"        : ['-',ang['kh_time']['lead_knee_extension_velocity']],
                    "at Fooc Contact"     : ['-41.50 ± 95.01',ang['fc_time']['lead_knee_extension_velocity']],
                    "at Ball Contact"     : ['252.48 ± 165.60',ang['bc_time']['lead_knee_extension_velocity']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Extension Angular Velocity',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro","Me"],
                    "Max"        : ['611.18 ± 372.09',ang['max']['lead_knee_extension_velocity']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['lead_knee_extension_velocity'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')

    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_lead_knee_ext_vel, ln = True, border='True')
    
    # ===========================================================================================================
    # Lead Elbow Extension Velcoity
    pdf.add_page()
    pdf.set_link(ch14_link, page = 15)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD ELBOW EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Sagittal plane angular velocity of the shank relative to the thigh')
    pdf.image('figure/LEAD ELBOW EXTENSION VELOCITY.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_vel.png',x=8,y=25,w=65,h=60)

    data_as_dict = {"":["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-70.52 ± 100.89',ang['to_time']['lead_elbow_flexion_velocity']],
                    "at Knee High"        : ['-',ang['kh_time']['lead_elbow_flexion_velocity']],
                    "at Fooc Contact"     : ['-41.50 ± 95.01',ang['fc_time']['lead_elbow_flexion_velocity']],
                    "at Ball Contact"     : ['252.48 ± 165.60',ang['bc_time']['lead_elbow_flexion_velocity']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Extension Angular Velocity',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"":["Pro","Me"],
                    "Max"        : ['611.18 ± 372.09',ang['max']['lead_elbow_flexion_velocity']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['lead_elbow_flexion_velocity'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')

    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_lead_elb_ext_vel, ln = True, border='True')
    # =============================================================================
    # Trunk Lataral Tilt
    pdf.add_page()
    pdf.set_link(ch15_link, page = 16)
    pdf.set_text_color(255,255,255)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7, 'SWING PHASE', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.cell(0,4,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK LATERAL TILT', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Frontal plane rotation of the trunk relative to the pelvis')
    pdf.image('figure/TRUNK LATERAL TILT.png',x=75,y=32,w=133,h=50)
    pdf.image(f'{logo_path}/lateral tilt.png',x=8,y=32,w=65,h=50)

    data_as_dict = {""                    : ["Pro","Me"],
                    "at Lead Leg Toe Off" : ['-4.77 ± 5.49', ang['to_time']['trunk_lateral_flexion']],
                    "at Knee High"        : ['-',ang['kh_time']['trunk_lateral_flexion']],
                    "at Fooc Contact"     : ['-8.17 ± 4.64',ang['fc_time']['trunk_lateral_flexion']],
                    "at Ball Contact"     : ['32.23 ± 6.40',ang['bc_time']['trunk_lateral_flexion']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lateral Trunk Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    data_as_dict = {"" :["Pro", "Me"],
                    "Max"        : ['34.77 ± 6.29',ang['max']['trunk_lateral_flexion']],
                    "Max Timing [BC - FC (100%)]" : ["-",f"{round(100 * (ang['max_time']['trunk_lateral_flexion'] - uplift_fc_time) / uplift_total_time,2)} %"],
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [66.6666,66.6666,66.6666],
                    x_start='C')
    
    pdf.set_y(158)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'Comments', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 9)
    pdf.set_y(163)
    pdf.multi_cell(0, 8, comment_trunk, ln = True, border='True')

    # =====================================================================================

    figure_file = glob('figure/*png')
    for figure in figure_file:
        os.remove(figure)
    
    pdf.output(f'{store}/{day}_{player}_baseball_report.pdf')

100%|██████████| 4/4 [00:52<00:00, 13.22s/it]
